In [2]:
import os
import sys
import re
import yaml
from math import floor
from itertools import combinations

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
from scipy import signal
from scipy.stats import norm, wilcoxon, mannwhitneyu, pearsonr, ttest_rel
from scipy.optimize import Bounds, LinearConstraint, minimize, SR1
from sklearn.metrics import roc_curve, auc
from bs4 import BeautifulSoup
from joypy import joyplot
from IPython.display import display, HTML

import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
from statsmodels.stats.multitest import multipletests



In [8]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import pandas as pd
import numpy as np
from IPython.display import display, HTML


from sklearn.metrics import roc_curve, auc

from pymer4.models import Lmer

from scipy import stats
from scipy.stats import norm, mannwhitneyu, ttest_rel
import statsmodels.stats.multitest as smm

import matplotlib.pyplot as plt
import seaborn as sns
import ptitprince as pt




### Load Behavioural and Similarity Data

In [ ]:
# BEHAVIOURAL DATA
folder_path = ''  # Update with your folder path
save_dir = './'
# Load all TSV files from the specified directory
data = []
file_names = []  # To store the participant IDs based on filenames
for filename in os.listdir(folder_path):
    if filename.endswith("gastric_stethoscope.tsv"):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path, sep='\t', header=1)  # Skip the first row (header)

        # Filter rows where 'Trial' is not numeric
        df = df[df['Trial'].str.isnumeric()]  # Keep only rows where 'Trial' is numeric
        df['Trial'] = df['Trial'].astype(int)  # Convert 'Trial' to integers
        df['Participant_Sound_Loc'] = df['Participant_Sound_Loc'].astype(int)
        df['Response_code'] = df['Response_code'].astype(int)
        df['Confidence'] = df['Confidence'].astype(int)
        
        # Exclude rows above the first instance where Trial = 0
        first_trial_zero_idx = df[df['Trial'] == 0].index.min()
        if not pd.isna(first_trial_zero_idx):
            df = df.loc[first_trial_zero_idx:]

        data.append(df)

        # Extract the participant ID from the filename (assuming the format "sub-XXX_gastric_stethascope.tsv")
        participant_id = filename.split('_')[0]  # Get the part before the first underscore (sub-XXX)
        file_names.append(participant_id)

# Concatenate all data into a single DataFrame
df = pd.concat(data, ignore_index=True)

cleaned_file_names = [name.replace('sub-', '') for name in file_names]
df['Participant'] = np.repeat(cleaned_file_names, [len(data[i]) for i in range(len(data))])
df["Participant"] = "sub-" + df["Participant"]

#Create accuracy column:
df['Accuracy'] = (df['Participant_Sound_Loc'] == df['Response_code']).astype(int)


# **Add Condition Column Based on Trial Number**
conditions = {
    (0, 9): 'fasted',
    (10, 19): 'sparkling_water',
    (20, 29): 'protein_shake'
}

def assign_condition(trial):
    for (start, end), condition in conditions.items():
        if start <= trial <= end:
            return condition
    return 'unknown'  # If outside expected trial ranges

df['Condition'] = df['Trial'].apply(assign_condition)


#display(HTML(df.head().to_html()))

### SIMILARITY DATA
similarity_df = pd.read_csv('Trial_by_trial_similarity.csv')
similarity_df.rename(columns={"participant_id": "Participant"}, inplace=True)
#display(HTML(similarity_df.head().to_html()))

### MERGE INTO A SINGLE DF
all_trial_by_trial_df = pd.merge(df, similarity_df, on = ['Participant', 'Trial'])
display(HTML(all_trial_by_trial_df.to_html()))

,Trial,Participant_Sound_Loc,Button,Response_code,Confidence,Participant,Accuracy,Condition,SimilarityTime_Live_Live,SimilarityTime_Live_Control_Zeros,SimilarityTime_Live_Control_Noise,SimilarityTime_Rec_Live,SimilarityTime_Rec_Recplay
0,0,1,Sound 2,2,10,sub-004,0,fasted,0.0,inf,0.246908,0.290334,0.227368
1,1,1,Sound 1,1,1,sub-004,1,fasted,0.0,inf,0.248676,0.226077,0.233391
2,2,1,Sound 1,1,74,sub-004,1,fasted,0.0,inf,0.262942,0.248903,0.308897
3,3,2,Sound 2,2,88,sub-004,1,fasted,0.0,inf,0.271174,0.276290,0.384106
4,4,1,Sound 1,1,92,sub-004,1,fasted,0.0,inf,0.293914,0.300780,0.313706
5,5,1,Sound 1,1,89,sub-004,1,fasted,0.0,inf,0.306064,0.334903,0.994566
6,6,2,Sound 2,2,95,sub-004,1,fasted,0.0,inf,0.279656,0.258610,0.322296
7,7,2,Sound 2,2,98,sub-004,1,fasted,0.0,inf,0.290017,0.279454,0.290379
8,8,2,Sound 2,2,85,sub-004,1,fasted,0.0,inf,0.528224,0.735814,0.594591
9,9,2,Sound 2,2,100,sub-004,1,fasted,0.0,inf,0.280258,0.267690,0.289474


### ACCURACY: ### MIXED LM DOES NOT DO BINOMIAL CORRECTLY - DO NOT USE
Staged mixed linear models <- does sound CID influence accuracy
1. formula0 = "Accuracy ~ Condition"
2. formula1 = "Accuracy ~ Condition + SimilarityTime_Rec_Live"
3. formula2 = "Accuracy ~ Condition + SimilarityTime_Rec_Live + SimilarityTime_Rec_Recplay"
4. formula3 = "Accuracy ~ Condition + SimilarityTime_Rec_Live + SimilarityTime_Rec_Recplay + SimilarityTime_Rec_Live:SimilarityTime_Rec_Recplay"

In [13]:
# Ensure Participant is treated as a categorical variable
all_trial_by_trial_df['Condition'] = pd.Categorical(all_trial_by_trial_df['Condition'], categories=['fasted', 'sparkling_water', 'protein_shake'], ordered=True)
all_trial_by_trial_df['Participant'] = pd.Categorical(all_trial_by_trial_df['Participant'])


# Fit the model
md = Lmer("Accuracy ~ Condition + SimilarityTime_Rec_Live + SimilarityTime_Rec_Recplay + SimilarityTime_Rec_Live:SimilarityTime_Rec_Recplay + (1|Participant)", data=all_trial_by_trial_df)
md.fit(factors = {"Condition": ['fasted','sparkling_water', 'protein_shake']})
print(md.summary())




Linear mixed model fit by REML [’lmerMod’]
Formula: Accuracy~Condition+SimilarityTime_Rec_Live+SimilarityTime_Rec_Recplay+SimilarityTime_Rec_Live:SimilarityTime_Rec_Recplay+(1|Participant)

Family: gaussian	 Inference: parametric

Number of observations: 1350	 Groups: {'Participant': 45.0}

Log-likelihood: -961.041 	 AIC: 1938.081

Random effects:

                    Name    Var    Std
Participant  (Intercept)  0.014  0.120
Residual                  0.229  0.478

No random effect correlations specified

Fixed effects:

Linear mixed model fit by REML [’lmerMod’]
Formula: Accuracy~Condition+SimilarityTime_Rec_Live+SimilarityTime_Rec_Recplay+SimilarityTime_Rec_Live:SimilarityTime_Rec_Recplay+(1|Participant)

Family: gaussian	 Inference: parametric

Number of observations: 1350	 Groups: {'Participant': 45.0}

Log-likelihood: -961.041 	 AIC: 1938.081

Random effects:

                    Name    Var    Std
Participant  (Intercept)  0.014  0.120
Residual                  0.229  0.478

No ra

c:\Users\hsavage\Documents\GitHub\Em-Body_toolkit-1\myenv\lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


### Check effect of Trial on Accuracy

In [16]:
# Ensure Participant is treated as a categorical variable
all_trial_by_trial_df['Condition'] = pd.Categorical(all_trial_by_trial_df['Condition'], categories=['fasted', 'sparkling_water', 'protein_shake'], ordered=True)
all_trial_by_trial_df['Participant'] = pd.Categorical(all_trial_by_trial_df['Participant'])


# Fit the model
md = Lmer("Accuracy ~ Trial + (1|Participant)", data=all_trial_by_trial_df)
md.fit()
print(md.summary())




Linear mixed model fit by REML [’lmerMod’]
Formula: Accuracy~Trial+(1|Participant)

Family: gaussian	 Inference: parametric

Number of observations: 1350	 Groups: {'Participant': 45.0}

Log-likelihood: -952.160 	 AIC: 1912.320

Random effects:

                    Name    Var    Std
Participant  (Intercept)  0.015  0.121
Residual                  0.229  0.479

No random effect correlations specified

Fixed effects:

Linear mixed model fit by REML [’lmerMod’]
Formula: Accuracy~Trial+(1|Participant)

Family: gaussian	 Inference: parametric

Number of observations: 1350	 Groups: {'Participant': 45.0}

Log-likelihood: -952.160 	 AIC: 1912.320

Random effects:

                    Name    Var    Std
Participant  (Intercept)  0.015  0.121
Residual                  0.229  0.479

No random effect correlations specified

Fixed effects:

             Estimate  2.5_ci  97.5_ci     SE        DF  T-stat  P-val  Sig
(Intercept)     0.551   0.490    0.612  0.031   163.672  17.663  0.000  ***
Trial   

c:\Users\hsavage\Documents\GitHub\Em-Body_toolkit-1\myenv\lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


### CONFIDENCE: 
Staged mixed linear models <- does sound CID influence confidence + accuracy relationship
1. formula0 = "Confidence ~ Condition"
2. formula1 = "Confidence ~ Condition + SimilarityTime_Rec_Live"
3. formula2 = "Confidence ~ Condition + SimilarityTime_Rec_Live + SimilarityTime_Rec_Recplay"
4. formula3 = "Confidence ~ Condition + SimilarityTime_Rec_Live + SimilarityTime_Rec_Recplay + SimilarityTime_Rec_Live:SimilarityTime_Rec_Recplay"

In [12]:
# Ensure Participant is treated as a categorical variable
all_trial_by_trial_df['Participant'] = all_trial_by_trial_df['Participant'].astype('category')

all_trial_by_trial_df['Condition'] = pd.Categorical(all_trial_by_trial_df['Condition'], categories=['fasted', 'sparkling_water', 'protein_shake'], ordered=True)
all_trial_by_trial_df['Participant'] = pd.Categorical(all_trial_by_trial_df['Participant'])


# Fit the model
md = Lmer("Confidence ~ Condition + SimilarityTime_Rec_Live + SimilarityTime_Rec_Recplay + SimilarityTime_Rec_Live:SimilarityTime_Rec_Recplay + (1|Participant)", data=all_trial_by_trial_df)
md.fit(factors = {"Condition": ['fasted','sparkling_water', 'protein_shake']})
print(md.summary())




Linear mixed model fit by REML [’lmerMod’]
Formula: Confidence~Condition+SimilarityTime_Rec_Live+SimilarityTime_Rec_Recplay+SimilarityTime_Rec_Live:SimilarityTime_Rec_Recplay+(1|Participant)

Family: gaussian	 Inference: parametric

Number of observations: 1350	 Groups: {'Participant': 45.0}

Log-likelihood: -5921.946 	 AIC: 11859.892

Random effects:

                    Name      Var     Std
Participant  (Intercept)  369.575  19.224
Residual                  339.174  18.417

No random effect correlations specified

Fixed effects:

Linear mixed model fit by REML [’lmerMod’]
Formula: Confidence~Condition+SimilarityTime_Rec_Live+SimilarityTime_Rec_Recplay+SimilarityTime_Rec_Live:SimilarityTime_Rec_Recplay+(1|Participant)

Family: gaussian	 Inference: parametric

Number of observations: 1350	 Groups: {'Participant': 45.0}

Log-likelihood: -5921.946 	 AIC: 11859.892

Random effects:

                    Name      Var     Std
Participant  (Intercept)  369.575  19.224
Residual              

c:\Users\hsavage\Documents\GitHub\Em-Body_toolkit-1\myenv\lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


### Check effect of Trial on Confidence

In [17]:
# Ensure Participant is treated as a categorical variable
all_trial_by_trial_df['Condition'] = pd.Categorical(all_trial_by_trial_df['Condition'], categories=['fasted', 'sparkling_water', 'protein_shake'], ordered=True)
all_trial_by_trial_df['Participant'] = pd.Categorical(all_trial_by_trial_df['Participant'])


# Fit the model
md = Lmer("Confidence ~ Trial + (1|Participant)", data=all_trial_by_trial_df)
md.fit()
print(md.summary())




Linear mixed model fit by REML [’lmerMod’]
Formula: Confidence~Trial+(1|Participant)

Family: gaussian	 Inference: parametric

Number of observations: 1350	 Groups: {'Participant': 45.0}

Log-likelihood: -5942.210 	 AIC: 11892.420

Random effects:

                    Name      Var     Std
Participant  (Intercept)  377.745  19.436
Residual                  347.166  18.632

No random effect correlations specified

Fixed effects:

Linear mixed model fit by REML [’lmerMod’]
Formula: Confidence~Trial+(1|Participant)

Family: gaussian	 Inference: parametric

Number of observations: 1350	 Groups: {'Participant': 45.0}

Log-likelihood: -5942.210 	 AIC: 11892.420

Random effects:

                    Name      Var     Std
Participant  (Intercept)  377.745  19.436
Residual                  347.166  18.632

No random effect correlations specified

Fixed effects:

             Estimate  2.5_ci  97.5_ci     SE        DF  T-stat  P-val  Sig
(Intercept)    36.378  30.378   42.379  3.062    51.635  1

c:\Users\hsavage\Documents\GitHub\Em-Body_toolkit-1\myenv\lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


### CONFIDENCE + ACCURACY: 
Staged mixed linear models <- does sound CID influence confidence + accuracy relationship
1. formula0 = "Confidence ~ Accuracy"
2. formula1 = "Confidence ~ Accuracy + SimilarityTime_Rec_Live"
3. formula2 = "Confidence ~ Accuracy + SimilarityTime_Rec_Live + SimilarityTime_Rec_Recplay"
4. formula3 = "Confidence ~ Accuracy + SimilarityTime_Rec_Live + SimilarityTime_Rec_Recplay + SimilarityTime_Rec_Live:SimilarityTime_Rec_Recplay"

In [18]:
# Ensure Participant is treated as a categorical variable
all_trial_by_trial_df['Participant'] = all_trial_by_trial_df['Participant'].astype('category')

all_trial_by_trial_df['Condition'] = pd.Categorical(all_trial_by_trial_df['Condition'], categories=['fasted', 'sparkling_water', 'protein_shake'], ordered=True)
all_trial_by_trial_df['Participant'] = pd.Categorical(all_trial_by_trial_df['Participant'])


# Fit the model
md = Lmer("Confidence ~ Accuracy + Condition + SimilarityTime_Rec_Live + SimilarityTime_Rec_Recplay + SimilarityTime_Rec_Live:SimilarityTime_Rec_Recplay + (1|Participant)", data=all_trial_by_trial_df)
md.fit(factors = {"Condition": ['fasted','sparkling_water', 'protein_shake']})
print(md.summary())




Linear mixed model fit by REML [’lmerMod’]
Formula: Confidence~Accuracy+Condition+SimilarityTime_Rec_Live+SimilarityTime_Rec_Recplay+SimilarityTime_Rec_Live:SimilarityTime_Rec_Recplay+(1|Participant)

Family: gaussian	 Inference: parametric

Number of observations: 1350	 Groups: {'Participant': 45.0}

Log-likelihood: -5901.877 	 AIC: 11821.755

Random effects:

                    Name      Var     Std
Participant  (Intercept)  360.151  18.978
Residual                  329.895  18.163

No random effect correlations specified

Fixed effects:

Linear mixed model fit by REML [’lmerMod’]
Formula: Confidence~Accuracy+Condition+SimilarityTime_Rec_Live+SimilarityTime_Rec_Recplay+SimilarityTime_Rec_Live:SimilarityTime_Rec_Recplay+(1|Participant)

Family: gaussian	 Inference: parametric

Number of observations: 1350	 Groups: {'Participant': 45.0}

Log-likelihood: -5901.877 	 AIC: 11821.755

Random effects:

                    Name      Var     Std
Participant  (Intercept)  360.151  18.978
Resi

c:\Users\hsavage\Documents\GitHub\Em-Body_toolkit-1\myenv\lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


In [14]:

import statsmodels.api as sm
from statsmodels.formula.api import glm

formula = "Confidence ~ SimilarityTime_Rec_Live + SimilarityTime_Rec_Recplay + SimilarityTime_Rec_Live:SimilarityTime_Rec_Recplay"

# Fit the model
model = glm(formula, data=all_trial_by_trial_df, family=sm.families.Gaussian()).fit()

# Print the model summary
print(model.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:             Confidence   No. Observations:                 1350
Model:                            GLM   Df Residuals:                     1346
Model Family:                Gaussian   Df Model:                            3
Link Function:               Identity   Scale:                          701.71
Method:                          IRLS   Log-Likelihood:                -6337.2
Date:                Wed, 02 Apr 2025   Deviance:                   9.4450e+05
Time:                        18:11:04   Pearson chi2:                 9.45e+05
No. Iterations:                     3   Pseudo R-squ. (CS):            0.02510
Covariance Type:            nonrobust                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [13]:
import statsmodels.api as sm
from statsmodels.formula.api import glm

formula = "Confidence ~ Accuracy + SimilarityTime_Rec_Live + SimilarityTime_Rec_Recplay + SimilarityTime_Rec_Live:SimilarityTime_Rec_Recplay"

# Fit the model
model = glm(formula, data=all_trial_by_trial_df, family=sm.families.Gaussian()).fit()

# Print the model summary
print(model.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:             Confidence   No. Observations:                 1350
Model:                            GLM   Df Residuals:                     1345
Model Family:                Gaussian   Df Model:                            4
Link Function:               Identity   Scale:                          681.47
Method:                          IRLS   Log-Likelihood:                -6316.9
Date:                Wed, 02 Apr 2025   Deviance:                   9.1657e+05
Time:                        18:10:51   Pearson chi2:                 9.17e+05
No. Iterations:                     3   Pseudo R-squ. (CS):            0.05497
Covariance Type:            nonrobust                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------